In [ ]:
%matplotlib notebook
import mne
import matplotlib.pyplot as plt

fname = "oddball_example-raw.fif"

# Read in raw data; raw objects

In [ ]:
raw = mne.io.read_raw_fif(fname)

MNE is *object oriented*. Objects have corresponding methods. Check which by typing `raw.` and pressing TAB:

In [ ]:
raw.

    raw.resample
    raw.filter
    raw.drop_channels
    ...

Can we do further preprocessing?..

In [ ]:
raw.filter(1, 20)

Aha! By default, MNE does not store raw and epochs objects in memory.

In [ ]:
raw = mne.io.read_raw_fif(fname, preload=True)

In [ ]:
raw.filter(1, 20)

Inspecting raw data ...

In [ ]:
raw.plot();

There are many eog artefacts. We will use ICA to correct these. For this, we create an ICA object
and use its `.fit` method on a filtered copy of the raw data:

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=0)

In [ ]:
ica.fit(raw.copy().filter(8, 35))

In [ ]:
ica.plot_components(outlines="skirt");

We store "bad" components in the ica object.

In [ ]:
ica.exclude = [0, 9, 14, 15, 17, 18, 19]

Let's compare raw and corrected data ...

In [ ]:
raw.plot();

In [ ]:
ica.apply(raw.copy(), exclude=ica.exclude).plot();

## Epochs

For epoching the data, we need event markers. Usually, these are stored in the `raw` object;
in MNE, in a stimulus channel.

In [ ]:
events = mne.find_events(raw)

`events` is simply an array (time in samples, zero, trigger);

In [ ]:
events

In [ ]:
events[:, 0] += 25

In [ ]:
mne.viz.plot_events(events[:100]);

For creating an `mne.Epochs` object, we require, in addition to the `raw` object and the `events` array, a dictionary of the intended condition names and the corresponding trigger numbers.

In [ ]:
event_ids = {"standard/stimulus": 200, "target/stimulus": 100}

epochs = mne.Epochs(raw, events, event_id=event_ids)

In [ ]:
epochs.plot();

In [ ]:
%matplotlib inline

In [ ]:
epochs = ica.apply(epochs, exclude=ica.exclude)

Of course ...

In [ ]:
epochs = mne.Epochs(raw, events, event_id=event_ids, preload=True)
epochs = ica.apply(epochs, exclude=ica.exclude)

The `mne.Epochs` constructor has a number of options, such as time window lengths and rejection thresholds.
Investigate them on your own.

`Epochs` objects also have various methods, different from `raw` objects - e.g., for baselining.

In [ ]:
epochs.apply_baseline((None, 0))

... and many more ...

In [ ]:
epochs.

To subselect only a sample of epochs, a dict-like access mode is available.

In [ ]:
epochs["target"]

Observe how tags selected by forward slashes - "/" - work.

In [ ]:
epochs["stimulus"]

In [ ]:
epochs["target"].plot_image(picks=[13]);

## Evokeds

Finally, if we average an epoched dataset over trials, we can use the `mne.Evoked` object.

In [ ]:
target = epochs["target"].average()
target

In [ ]:
standard = epochs["standard"].average()

To quickly investigate evoked activity, the `Evoked` object has a number of plotting functions available.

In [ ]:
target.plot_joint();

For condition contrasts, you can use `mne.combine.evoked`:

In [ ]:
mne.combine_evoked((target, standard), weights=(.5, -.5)).plot_joint(times=.35);

To contrast multiple conditions, `mne.viz.plot_compare_evokeds` is available:

In [ ]:
mne.viz.plot_compare_evokeds({"target": target, "standard": standard},
                             picks=[21, 22, 23], truncate_yaxis=True);

## MVPA/decoding

Can we predict trial type from EEG activity?

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

To speed up and simplify things, we balance the epochs counts for the two trials.

In [ ]:
epochs.equalize_event_counts(event_ids);

In [ ]:
epochs

In [ ]:
epochs.pick_types(eeg=True)
X = epochs.get_data()  # features
y = epochs.events[:, -1] == event_ids["target/stimulus"]  # targets
X.shape, y.shape

X has the wrong shape - `samples`, `channels`, `times`, should be `samples`, `features`.

We can use `mne.decoding.Vectorizer` to correctly shape the data. It fits right into a scikit-learn pipeline.

In [ ]:
from mne.decoding import Vectorizer
clf = make_pipeline(Vectorizer(), StandardScaler(), LinearSVC(class_weight="balanced"))

In [ ]:
cross_val_score(clf, X, y)  # accuracy

At which time points in the trial is there information about trial category?

We need two more tools for this: one to train and score at each time point, and one to handle the cross-validated scoring for the former.

In [ ]:
from mne.decoding import SlidingEstimator, cross_val_multiscore
sl = SlidingEstimator(clf)

In [ ]:
scores_time_decoding = cross_val_multiscore(sl, X, y)

In [ ]:
scores_time_decoding.shape

In [ ]:
fig, ax = plt.subplots()
ax.plot(epochs.times, scores_time_decoding.mean(0))
plt.show()

But is the same thing happening at each time point? We can investigate that with generalization across time decoding.

In [ ]:
from mne.decoding import GeneralizingEstimator
gen = GeneralizingEstimator(clf)
scores_gat = cross_val_multiscore(gen, X, y)

In [ ]:
scores_gat.shape

In [ ]:
import numpy as np
data = scores_gat.mean(0)
vmax = np.abs(data).max()
tmin, tmax = epochs.times[[0, -1]]

fig, ax = plt.subplots()
im = ax.imshow(
    data,
    origin="lower", cmap="RdBu_r",
    extent=(tmin, tmax, tmin, tmax),
    vmax=vmax, vmin=1-vmax);

plt.colorbar(im)

... also see Jona's talk:


**Talk session: Language** (02/0090)

**17:45 - Decoding the P600: MVPA evidence for shared neural patterns underlying Oddball and syntactic violation processing**

        Jona Sassenhagen & Christian J. Fiebach

Dejan's poster:

**Perception II**

**T-1-22 - Investigating the temporal dynamics of object-scene integration using MVPA: The role of the N300/N400 complex in object perception**

        Dejan Draschkow, Edvard Aslak Heikel, Melissa L.-H. Võ, Christian J Fiebach & Jona Sassenhagen

And:

**Language**

**T-1-15 - Inability to decode predictable semantic categories from EEG during silent pauses in spoken language**

        Edvard Aslak Heikel, Jona Sassenhagen & Christian J. Fiebach
        



## Time-Frequency stuff

For an overview over the spectral shape of the data, we can use a plotting method of `raw`, `raw.plot_psd`:

In [ ]:
raw.plot_psd(fmin=1, fmax=20);

But what about the time/frequency correlates of the Oddball effect?

We will extract power per time and frequency with Morlet wavelets.

In [ ]:
from mne.time_frequency import tfr_morlet

In [ ]:
epochs_for_tfr = mne.Epochs(raw, events, event_id=event_ids,
                            tmin=-.5, tmax=1.5, preload=True)  # need longer data segment
epochs_for_tfr = ica.apply(epochs_for_tfr, exclude=ica.exclude)
epochs_for_tfr.equalize_event_counts(event_ids);  # to speed up things

In [ ]:
freqs = np.arange(3, 30)
tfr_target = tfr_morlet(epochs_for_tfr["target"], freqs, 3, return_itc=False)
tfr_standard = tfr_morlet(epochs_for_tfr["standard"], freqs, 3, return_itc=False)

Time-frequency data (single trial or averaged) is stored in TFR objects. These objects behave in many ways like Evoked objects ...

In [ ]:
tfr = mne.combine_evoked((tfr_standard, tfr_target), (-.5, .5))
tfr.apply_baseline((None, 0))

Plotting time-frequencyy activity (event-related spectral perturbations): observe the alpha-band ERD and the time-frequency correlates of the P3 effect.

In [ ]:
tfr.plot(picks=[27]);

## Statistics

### Cluster-based permutation stats

Exploratory analysis with nonparametric control of the error rate is commonly done with
cluster-based permutation tests (i.e., Maris 2012). To cluster across space, we first need a
channel adjacency matrix.

In [ ]:
from mne.channels import find_ch_connectivity
connectivity, ch_names = find_ch_connectivity(epochs.info, ch_type='eeg')
plt.imshow(connectivity.toarray(), cmap="Greys")

Now we need the data in the right shape. Sadly, because the space dimension needs
to be last, we need to manually swap the time and space axes.

In [ ]:
epochs.pick_types(eeg=True)
target_epochs, standard_epochs = epochs["target"].get_data(), epochs["standard"].get_data()
target_epochs.shape, standard_epochs.shape

In [ ]:
target_epochs = target_epochs.swapaxes(1, 2)
standard_epochs = standard_epochs.swapaxes(1, 2)
target_epochs.shape, standard_epochs.shape

MNE has various cluster-based permutation test options. Here, we test for single-trial
differences between conditions with `mne.stats.spatio_temporal_cluster_test`.

We use threshold-free cluster enhancement to reduce the number of parameters.

Warning: the next cell takes a lot of time and computational power.

In [ ]:
from mne.stats import spatio_temporal_cluster_test

tfce = dict(start=.1, step=.3)
cluster_stats = spatio_temporal_cluster_test([target_epochs, standard_epochs],
                                             threshold=tfce,
                                             n_permutations=256,
                                             n_jobs=-1,
                                             connectivity=connectivity)
T_obs, clusters, p_values, _ = cluster_stats

Now we can visualise the *t* values over time and space ...

In [ ]:
extent = (*epochs.times[[0, -1]], 0, len(epochs.ch_names))
im = plt.imshow(T_obs.T, aspect="auto", cmap="RdBu_r",
                vmin=-100, vmax=100, extent=extent
          )
plt.colorbar(im)

... and the p-values.

In [ ]:
plt.hist(p_values)

alpha = .01
print(sum(p_values < alpha))

In [ ]:
plt.imshow(p_values.reshape(T_obs.shape).T < alpha, aspect="auto", cmap="Reds",
           extent=extent
          )

### Parametric stats
Sometimes, e.g. because we wish to test a specific hypothesis, cluster-based permutation tests are too much.
We can also simply access the data in array form and test with parametric (or nonparametric) tests.

For this, we first need to identify the spatial and temporal coordinates of an effect we want to test -
for example, the N2 at Cz.

In [ ]:
time_mask = (.2 < epochs.times) & (epochs.times < .25)
electrode_pz = epochs.ch_names.index("Cz")
plt.plot(time_mask)

Now we extract the target data. Reminder: the shape of epochs data is (trial, channel, time)

In [ ]:
epochs["target"].get_data().shape

In [ ]:
cond_a = epochs["target"].get_data()[:, electrode_pz, time_mask].mean(-1)
cond_b = epochs["standard"].get_data()[:, electrode_pz, time_mask].mean(-1)

Now we can simply use ordinary tests on these statistics.

In [ ]:
from scipy.stats import ttest_ind, wilcoxon

In [ ]:
ttest_ind(cond_a, cond_b)

In [ ]:
wilcoxon(cond_a, cond_b)

It is also straight-forward to convert the data into a (pandas) dataframe.

In [ ]:
df = epochs.to_data_frame()
df.head(20)

In [ ]:
df_cz = df.query("200 < time < 250")["Cz"].groupby(["epoch", "condition"]).mean().reset_index()
df_cz.head()

In [ ]:
import seaborn as sns
sns.factorplot(y="Cz", data=df_cz, x="condition")